In [ ]:
import anndata

import os
import numpy
from aavomics import database

from plotly import graph_objects
from plotly import offline as plotly

In [ ]:
ANNDATA_FILE_NAME = "aavomics_mouse_cortex_2021.h5ad"
CELL_SET_NAME = "20190711_TC4"
VIRUS_NAME = "CAP-B10"

In [ ]:
CELL_TYPE_HIERARCHY = {
    "Astrocytes": {
        "Myoc- Astrocytes": {},
        "Myoc+ Astrocytes": {}
    },
    "Vascular Cells": {
        "Endothelial Cells": {},
        "Pericytes": {},
        "Red Blood Cells": {},
        "Vascular SMCs": {},
        "VLMCs": {}
    },
    "Immune Cells": {
        "Perivascular Macrophages": {},
        "Microglia": {},
        "Leukocytes": {}
    },
    "Oligodendrocytes": {
        "OPCs": {},
        "Committed Oligodendrocytes": {},
        "Mature Oligodendrocytes": {}
    },
    "Neurons": {
        "L2": {},
        "L2/3": {},
        "L3": {},
        "L4/5": {},
        "L5": {},
        "L5/6": {},
        "L6": {},
        "Lamp5": {},
        "Pax6": {},
        "Sncg": {},
        "Vip": {},
        "Sst": {},
        "Pvalb": {}
    }
}

In [ ]:
adata = anndata.read_h5ad(os.path.join(database.DATA_PATH, ANNDATA_FILE_NAME))
cell_set_adata = adata[adata.obs["Cell Set"] == CELL_SET_NAME].copy()

In [ ]:
traces = []
max_y_value = -numpy.infty

for cell_type_name in CELL_TYPE_HIERARCHY:

    cell_type_mask = cell_set_adata.obs["Cell Type"].isin(list(CELL_TYPE_HIERARCHY[cell_type_name].keys()))
    
    cell_type_adata = cell_set_adata[cell_type_mask].copy()
    
    num_cells = cell_type_mask.sum()

    x_values = []
    y_values = []

    for threshold in range(50):

        num_above_threshold = (cell_type_adata.obs[VIRUS_NAME].values > threshold).sum()
        transduction_rate = num_above_threshold/num_cells

        x_values.append(threshold+1)
        y_values.append(transduction_rate*100)

    trace = graph_objects.Scatter(
        x=x_values,
        y=y_values,
        name=cell_type_name,
        mode="lines+markers"
    )

    max_y_value = max(max_y_value, max(y_values))

    traces.append(trace)

layout = {
    "xaxis": {
        "title": "Transcript Count Threshold",
        "zerolinecolor": "rgba(0, 0, 0, 1)",
        "range": [0, 30]
    },
    "yaxis": {
        "title": "% of Cells Expressing Transcripts Above Threshold",
        "gridcolor": "rgba(0, 0, 0, 0.25)",
        "zerolinecolor": "rgba(0, 0, 0, 1)",
        "range": [0, max_y_value*1.1]
    },
    "width": 600,
    "plot_bgcolor": "rgba(255, 255, 255, 0)",
    "paper_bgcolor": "rgba(255, 255, 255, 0)",
    "title": "%s %s" % (CELL_SET_NAME, VIRUS_NAME)
}


figure = graph_objects.Figure(data=traces, layout=layout)

plotly.iplot(figure)

figure.write_image(os.path.join("out", "%s_%s_threshold_transduction_rates.svg" % (CELL_SET_NAME, VIRUS_NAME)))